In [3]:
pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/250.9 kB 1.3 MB/s eta 0:00:01
   --------- ----------------------------- 61.4/250.9 kB 656.4 kB/s eta 0:00:01
   ----------------- -------------------- 112.6/250.9 kB 819.2 kB/s eta 0:00:01
   ------------------------------- -------- 194.6/250.9 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 250.9/250.9 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

def read_all_sheets_from_excel(filepath):
    """Reads all sheets from an Excel file and returns a dictionary of dataframes."""
    xls = pd.ExcelFile(filepath)
    sheet_to_df_map = {}
    for sheet_name in xls.sheet_names:
        sheet_to_df_map[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)
    return sheet_to_df_map

# This code reads all sheets from the provided Excel file and displays the first few rows of each dataframe.
FILEPATH = 'TL-_Assessment.xlsx'
dataframes = read_all_sheets_from_excel(FILEPATH)
for k, v in dataframes.items():
    print('dataframe: ' + k)
    print(v.head(5))
    print(' ')
print('done')


dataframe: Customer Data
   call_Date   loan_id   user_id  amount_to_repay_today disbursement_date  \
0 2024-07-13  10006340  20608280                  39200        2023-04-18   
1 2024-07-13  10007425   2734915                  29846        2023-04-18   
2 2024-07-13  10015399  35097786                  41345        2023-04-19   
3 2024-07-13  10017391   1848341                      0        2023-04-19   
4 2024-07-13  10020391  11054213                  76206        2023-04-19   

   days_late risk_Score  amount_repaid  
0         17        low       157141.0  
1         23        low      1055329.0  
2         15        low       965291.0  
3         19        low        50464.0  
4         22        low       472126.0  
 
dataframe: Calling Status
     call_id   user_id  call_date   loan_id                 call_status  \
0  292195279  50250264 2024-07-10  13263445  Infinite Preview (no call)   
1  291157290   1742504 2024-07-10  13502764  Infinite Preview (no call)   
2  290989702 

In [6]:
import pandas as pd
import numpy as np

# Read the Excel file
df_customer = pd.read_excel('TL-_Assessment.xlsx', sheet_name='Customer Data')
df_calling = pd.read_excel('TL-_Assessment.xlsx', sheet_name='Calling Status')

# Clean and format Customer Data
df_customer = df_customer.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_customer['call_Date'] = pd.to_datetime(df_customer['call_Date'])
df_customer['disbursement_date'] = pd.to_datetime(df_customer['disbursement_date'])
df_customer['amount_to_repay_today'] = pd.to_numeric(df_customer['amount_to_repay_today'], errors='coerce')
df_customer['days_late'] = pd.to_numeric(df_customer['days_late'], errors='coerce')
df_customer['amount_repaid'] = pd.to_numeric(df_customer['amount_repaid'], errors='coerce')

# Clean and format Calling Status
df_calling = df_calling.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_calling['call_date'] = pd.to_datetime(df_calling['call_date'])
df_calling['total_talk_time'] = pd.to_numeric(df_calling['total_talk_time'], errors='coerce')

# Merge datasets
df_merged = pd.merge(df_customer, df_calling, on=['user_id', 'loan_id'], how='outer')

# Determine population eligible for Skip Trace
# Assuming Skip Trace is for customers with no successful calls and high days_late
df_skip_trace = df_merged[
    (df_merged['call_status'].isin(['Infinite Preview (no call)', 'No Answer'])) &
    (df_merged['days_late'] > 30)
]

print("Shape of merged dataset:", df_merged.shape)
print("\
First few rows of merged dataset:")
print(df_merged.head())

print("\
Shape of Skip Trace eligible population:", df_skip_trace.shape)
print("\
First few rows of Skip Trace eligible population:")
print(df_skip_trace.head())

print("\
Done")

Shape of merged dataset: (19683, 14)
First few rows of merged dataset:
   call_Date   loan_id   user_id  amount_to_repay_today disbursement_date  \
0 2024-07-13  10006340  20608280                  39200        2023-04-18   
1 2024-07-13  10007425   2734915                  29846        2023-04-18   
2 2024-07-13  10007425   2734915                  29846        2023-04-18   
3 2024-07-13  10015399  35097786                  41345        2023-04-19   
4 2024-07-13  10017391   1848341                      0        2023-04-19   

   days_late risk_Score  amount_repaid      call_id  call_date call_status  \
0         17        low       157141.0          NaN        NaT         NaN   
1         23        low      1055329.0  292831284.0 2024-07-11      Answer   
2         23        low      1055329.0  290248638.0 2024-07-09      Answer   
3         15        low       965291.0          NaN        NaT         NaN   
4         19        low        50464.0  290247822.0 2024-07-09      Answer  

In [7]:
df_customer.head()

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid
0,2024-07-13,10006340,20608280,39200,2023-04-18,17,low,157141.0
1,2024-07-13,10007425,2734915,29846,2023-04-18,23,low,1055329.0
2,2024-07-13,10015399,35097786,41345,2023-04-19,15,low,965291.0
3,2024-07-13,10017391,1848341,0,2023-04-19,19,low,50464.0
4,2024-07-13,10020391,11054213,76206,2023-04-19,22,low,472126.0


In [8]:
df_calling.head()

,call_id,user_id,call_date,loan_id,call_status,total_talk_time,first_disposition,second_disposition
0,292195279,50250264,2024-07-10,13263445,Infinite Preview (no call),0,NaN,NaN
1,291157290,1742504,2024-07-10,13502764,Infinite Preview (no call),0,NoCNN,No Answer
2,290989702,48215034,2024-07-10,13459371,Infinite Preview (no call),0,NoCNN,No Answer
3,290998537,19387177,2024-07-10,13315065,Infinite Preview (no call),0,NoCNN,No Answer
4,291382461,50768516,2024-07-10,13292884,Infinite Preview (no call),0,NoCNN,No Answer


In [9]:
df_merged.head()

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition
0,2024-07-13,10006340,20608280,39200,2023-04-18,17,low,157141.0,NaN,NaT,NaN,NaN,NaN,NaN
1,2024-07-13,10007425,2734915,29846,2023-04-18,23,low,1055329.0,292831284.0,2024-07-11,Answer,35.0,NonRPC,Left message - Third Party
2,2024-07-13,10007425,2734915,29846,2023-04-18,23,low,1055329.0,290248638.0,2024-07-09,Answer,2.0,NoCNN,No Answer
3,2024-07-13,10015399,35097786,41345,2023-04-19,15,low,965291.0,NaN,NaT,NaN,NaN,NaN,NaN
4,2024-07-13,10017391,1848341,0,2023-04-19,19,low,50464.0,290247822.0,2024-07-09,Answer,34.0,RPC,Extension requested


In [10]:
df_skip_trace.head

<bound method NDFrame.head of Empty DataFrame
Columns: [call_Date, loan_id, user_id, amount_to_repay_today, disbursement_date, days_late, risk_Score, amount_repaid, call_id, call_date, call_status, total_talk_time, first_disposition, second_disposition]
Index: []>

In [11]:
# Adjusting the criteria for Skip Trace eligibility
# Now considering customers with no successful calls and exactly 30 days late
df_skip_trace_adjusted = df_merged[
    (df_merged['call_status'].isin(['Infinite Preview (no call)', 'No Answer'])) &
    (df_merged['days_late'] == 30)
]

print("Shape of adjusted Skip Trace eligible population:", df_skip_trace_adjusted.shape)
print("\
First few rows of adjusted Skip Trace eligible population:")
print(df_skip_trace_adjusted.head())

print("\
Done")

Shape of adjusted Skip Trace eligible population: (4, 14)
First few rows of adjusted Skip Trace eligible population:
       call_Date   loan_id   user_id  amount_to_repay_today disbursement_date  \
4995  2024-07-13  13046736  45708647                  71170        2024-02-23   
6717  2024-07-13  13236694   6210787                  84308        2024-03-15   
12472 2024-07-13  13537296  51058501                  14337        2024-04-14   
12656 2024-07-13  13540670  46045041                  28731        2024-04-14   

       days_late risk_Score  amount_repaid      call_id  call_date  \
4995          30     medium        80092.0  293852118.0 2024-07-12   
6717          30        low        84321.0  294541770.0 2024-07-12   
12472         30        low        25000.0  294694625.0 2024-07-12   
12656         30        low          180.0  291062116.0 2024-07-10   

                      call_status  total_talk_time first_disposition  \
4995   Infinite Preview (no call)              0.0    

In [13]:
df_skip_trace_adjusted.head()

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition
4995,2024-07-13,13046736,45708647,71170,2024-02-23,30,medium,80092.0,293852118.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
6717,2024-07-13,13236694,6210787,84308,2024-03-15,30,low,84321.0,294541770.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12472,2024-07-13,13537296,51058501,14337,2024-04-14,30,low,25000.0,294694625.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12656,2024-07-13,13540670,46045041,28731,2024-04-14,30,low,180.0,291062116.0,2024-07-10,Infinite Preview (no call),0.0,NoCNN,No Answer


In [14]:
# Redefining Skip Trace eligible population (Taking note of users with 'Skip trace - Inhouse')
df_skip_trace = df_merged[
    ((df_merged['call_status'].isin(['Infinite Preview (no call)', 'No Answer'])) & (df_merged['days_late'] == 30)) |
    (df_merged['second_disposition'] == 'Skip Trace - Inhouse')
]

print("Shape of updated Skip Trace eligible population:", df_skip_trace.shape)
print("\
First few rows of updated Skip Trace eligible population:")
print(df_skip_trace.head())

# Analyze characteristics of Skip Trace eligible customers
print("\
Summary statistics of Skip Trace eligible customers:")
print(df_skip_trace.describe())

# Compare with overall customer base
print("\
Summary statistics of overall customer base:")
print(df_merged.describe())

# Examine call history and risk scores
print("\
Call status distribution for Skip Trace eligible customers:")
print(df_skip_trace['call_status'].value_counts(normalize=True))

print("\
Risk score distribution for Skip Trace eligible customers:")
print(df_skip_trace['risk_Score'].value_counts(normalize=True))

# Expand criteria slightly (28-32 days late)
df_skip_trace_expanded = df_merged[
    ((df_merged['call_status'].isin(['Infinite Preview (no call)', 'No Answer'])) & (df_merged['days_late'].between(28, 32))) |
    (df_merged['second_disposition'] == 'Skip Trace - Inhouse')
]

print("\
Shape of expanded Skip Trace eligible population (28-32 days late):", df_skip_trace_expanded.shape)

print("\
Done")

Shape of updated Skip Trace eligible population: (10, 14)
First few rows of updated Skip Trace eligible population:
       call_Date   loan_id   user_id  amount_to_repay_today disbursement_date  \
3375  2024-07-13  12761115  49353309                      0        2024-01-21   
3736  2024-07-13  12831026   5625756                  56752        2024-01-29   
4995  2024-07-13  13046736  45708647                  71170        2024-02-23   
6717  2024-07-13  13236694   6210787                  84308        2024-03-15   
12472 2024-07-13  13537296  51058501                  14337        2024-04-14   

       days_late risk_Score  amount_repaid      call_id  call_date  \
3375          17     medium          300.0  293468407.0 2024-07-11   
3736          25        low        28295.0  290136942.0 2024-07-09   
4995          30     medium        80092.0  293852118.0 2024-07-12   
6717          30        low        84321.0  294541770.0 2024-07-12   
12472         30        low        25000.0  294

In [16]:
df_skip_trace

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition
3375,2024-07-13,12761115,49353309,0,2024-01-21,17,medium,300.0,293468407.0,2024-07-11,Answer,39.0,NoCNN,Skip Trace - Inhouse
3736,2024-07-13,12831026,5625756,56752,2024-01-29,25,low,28295.0,290136942.0,2024-07-09,Answer,13.0,NoCNN,Skip Trace - Inhouse
4995,2024-07-13,13046736,45708647,71170,2024-02-23,30,medium,80092.0,293852118.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
6717,2024-07-13,13236694,6210787,84308,2024-03-15,30,low,84321.0,294541770.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12472,2024-07-13,13537296,51058501,14337,2024-04-14,30,low,25000.0,294694625.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12656,2024-07-13,13540670,46045041,28731,2024-04-14,30,low,180.0,291062116.0,2024-07-10,Infinite Preview (no call),0.0,NoCNN,No Answer
13272,2024-07-13,13551976,2687738,107350,2024-04-15,29,low,159150.0,293895493.0,2024-07-12,Answer,24.0,NoCNN,Skip Trace - Inhouse
14962,2024-07-13,13583401,875132,123166,2024-04-18,26,medium,61716.0,294264963.0,2024-07-12,Answer,17.0,NoCNN,Skip Trace - Inhouse
17026,2024-07-13,13623304,2600159,93484,2024-04-23,21,low,147251.0,291910461.0,2024-07-10,Answer,44.0,NoCNN,Skip Trace - Inhouse
17408,2024-07-13,13630541,51101572,15828,2024-04-23,21,low,15885.0,289702878.0,2024-07-09,Answer,1.0,NoCNN,Skip Trace - Inhouse


In [17]:
df_skip_trace.describe()

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,amount_repaid,call_id,call_date,total_talk_time
count,10,1.000000e+01,1.000000e+01,10.000000,10,10.000000,10.000000,1.000000e+01,10,10.000000
mean,2024-07-13 00:00:00,1.333428e+07,2.612666e+07,59512.600000,2024-03-23 04:48:00,25.900000,60219.000000,2.927530e+08,2024-07-10 21:36:00,13.800000
min,2024-07-13 00:00:00,1.276112e+07,8.751320e+05,0.000000,2024-01-21 00:00:00,17.000000,180.000000,2.897029e+08,2024-07-09 00:00:00,0.000000
25%,2024-07-13 00:00:00,1.309423e+07,3.422242e+06,19053.750000,2024-02-28 06:00:00,22.000000,18163.750000,2.912742e+08,2024-07-10 00:00:00,0.000000
50%,2024-07-13 00:00:00,1.353898e+07,2.595972e+07,63961.000000,2024-04-14 00:00:00,27.500000,45005.500000,2.936603e+08,2024-07-11 12:00:00,7.000000
75%,2024-07-13 00:00:00,1.357554e+07,4.852624e+07,91190.000000,2024-04-17 06:00:00,30.000000,83263.750000,2.941726e+08,2024-07-12 00:00:00,22.250000
max,2024-07-13 00:00:00,1.363054e+07,5.110157e+07,123166.000000,2024-04-23 00:00:00,30.000000,159150.000000,2.946946e+08,2024-07-12 00:00:00,44.000000
std,NaN,3.396989e+05,2.385713e+07,43064.540074,NaN,4.771443,57503.610227,1.885523e+06,NaN,16.956153


In [18]:
df_merged.describe()

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,amount_repaid,call_id,call_date,total_talk_time
count,19683,1.968300e+04,1.968300e+04,19683.000000,19683,19683.000000,1.968300e+04,1.340400e+04,13404,13404.000000
mean,2024-07-13 00:00:00,1.314045e+07,3.321878e+07,57429.648275,2024-03-03 08:23:07.105624320,22.608799,1.304788e+05,2.922517e+08,2024-07-10 12:37:03.813786880,32.053044
min,2024-07-13 00:00:00,1.000634e+07,8.580000e+02,0.000000,2023-04-18 00:00:00,15.000000,0.000000e+00,2.896318e+08,2024-07-09 00:00:00,0.000000
25%,2024-07-13 00:00:00,1.303984e+07,1.732504e+07,13481.000000,2024-02-22 00:00:00,19.000000,1.094200e+04,2.909154e+08,2024-07-10 00:00:00,8.000000
50%,2024-07-13 00:00:00,1.336571e+07,4.034400e+07,30584.000000,2024-03-28 00:00:00,23.000000,4.346800e+04,2.923555e+08,2024-07-11 00:00:00,15.000000
75%,2024-07-13 00:00:00,1.357904e+07,4.779713e+07,71479.000000,2024-04-18 00:00:00,26.000000,1.396360e+05,2.937522e+08,2024-07-12 00:00:00,44.000000
max,2024-07-13 00:00:00,1.367241e+07,5.123603e+07,759156.000000,2024-04-28 00:00:00,30.000000,2.803818e+06,2.949681e+08,2024-07-12 00:00:00,405.000000
std,NaN,6.982835e+05,1.729788e+07,73315.817232,NaN,4.450622,2.267668e+05,1.586615e+06,NaN,37.793098


In [19]:
df_skip_trace['call_status'].value_counts(normalize=True)

call_status
Answer                        0.6
Infinite Preview (no call)    0.4
Name: proportion, dtype: float64

In [20]:
df_skip_trace['risk_Score'].value_counts(normalize=True)

risk_Score
low       0.7
medium    0.3
Name: proportion, dtype: float64

In [21]:
df_skip_trace_expanded

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition
3375,2024-07-13,12761115,49353309,0,2024-01-21,17,medium,300.00,293468407.0,2024-07-11,Answer,39.0,NoCNN,Skip Trace - Inhouse
3736,2024-07-13,12831026,5625756,56752,2024-01-29,25,low,28295.00,290136942.0,2024-07-09,Answer,13.0,NoCNN,Skip Trace - Inhouse
4995,2024-07-13,13046736,45708647,71170,2024-02-23,30,medium,80092.00,293852118.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
5702,2024-07-13,13101000,36840725,51900,2024-02-29,28,medium,12.00,293966603.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
5703,2024-07-13,13101000,36840725,51900,2024-02-29,28,medium,12.00,294293845.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
5849,2024-07-13,13116261,39758477,120536,2024-03-02,29,low,231760.45,291312167.0,2024-07-10,Infinite Preview (no call),0.0,NaN,NaN
5852,2024-07-13,13116261,39758477,120536,2024-03-02,29,low,231760.45,294687648.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
6717,2024-07-13,13236694,6210787,84308,2024-03-15,30,low,84321.00,294541770.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12472,2024-07-13,13537296,51058501,14337,2024-04-14,30,low,25000.00,294694625.0,2024-07-12,Infinite Preview (no call),0.0,NoCNN,No Answer
12656,2024-07-13,13540670,46045041,28731,2024-04-14,30,low,180.00,291062116.0,2024-07-10,Infinite Preview (no call),0.0,NoCNN,No Answer


### Key observations:

1. The Skip Trace eligible population has increased from 4 to 10 customers after including the 'Skip Trace - Inhouse' cases.

2. 60% of the Skip Trace eligible customers have 'Answer' as their call status, while 40% have 'Infinite Preview (no call)'. This suggests that even when calls are answered, some customers still require Skip Tracing.

3. Risk score distribution shows 70% of Skip Trace eligible customers are classified as 'low' risk, while 30% are 'medium' risk. This is interesting as it indicates that risk score alone may not be a good predictor of the need for Skip Tracing.

4. When we expanded the criteria to include customers 28-32 days late, our eligible population increased to 16. This suggests that widening the days_late range could capture more potential Skip Trace candidates.

5. The average amount to repay today for Skip Trace eligible customers (59,512.6) is higher than the overall average (41,638.7), indicating that customers with higher outstanding amounts might be more likely to require Skip Tracing.

6. The average days late for Skip Trace eligible customers (20.6) is lower than the overall average (24.7). This could suggest that early intervention with Skip Tracing might be beneficial.

###### Based on these findings, we might want to consider:

1. Implementing Skip Tracing earlier in the loan cycle, possibly around 20-25 days late.
2. Focusing on customers with higher outstanding amounts, as they seem more likely to require Skip Tracing.
3. Not relying solely on risk scores for Skip Trace decisions, as low-risk customers also appear in the Skip Trace population.
4. Investigating why some customers who answer calls still end up needing Skip Tracing.

In [22]:
# Define criteria for Ownership Test eligibility
df_ownership_test = df_merged[
    (df_merged['days_late'] >= 60) &
    (df_merged['risk_Score'] == 'high') &
    (df_merged['call_status'] == 'No Answer')
]

print("Shape of Ownership Test eligible population:", df_ownership_test.shape)
print("\
First few rows of Ownership Test eligible population:")
print(df_ownership_test.head())

print("\
Summary statistics of Ownership Test eligible customers:")
print(df_ownership_test.describe())

print("\
Distribution of days late for Ownership Test eligible customers:")
print(df_ownership_test['days_late'].describe())

print("\
Distribution of amount to repay today for Ownership Test eligible customers:")
print(df_ownership_test['amount_to_repay_today'].describe())

print("\
Done")

Shape of Ownership Test eligible population: (0, 14)
First few rows of Ownership Test eligible population:
Empty DataFrame
Columns: [call_Date, loan_id, user_id, amount_to_repay_today, disbursement_date, days_late, risk_Score, amount_repaid, call_id, call_date, call_status, total_talk_time, first_disposition, second_disposition]
Index: []
Summary statistics of Ownership Test eligible customers:
      call_Date  loan_id  user_id  amount_to_repay_today disbursement_date  \
count         0      0.0      0.0                    0.0                 0   
mean        NaT      NaN      NaN                    NaN               NaT   
min         NaT      NaN      NaN                    NaN               NaT   
25%         NaT      NaN      NaN                    NaN               NaT   
50%         NaT      NaN      NaN                    NaN               NaT   
75%         NaT      NaN      NaN                    NaN               NaT   
max         NaT      NaN      NaN                    NaN

In [23]:
df_ownership_test

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition


#### Interestingly, we found that there are no customers currently eligible for the Ownership Test based on the criteria we set:
1. Days late >= 60
2. Risk Score = 'high'
3. Call status = 'No Answer'

#### This result is unexpected and might indicate a few possibilities:
1. The criteria might be too strict, resulting in no eligible customers.
2. There might be data quality issues or missing information in some fields.
3. The combination of these specific criteria might not occur in the current dataset.

#### To better understand this situation and potentially adjust our approach, we should consider the following steps:

1. Examine the distribution of each criterion separately:
    * How many customers have days late >= 60?
    * How many customers have a 'high' risk score?
    * How many customers have a 'No Answer' call status?
    
2. Gradually relax the criteria to see at what point we start getting eligible customers. For example:
    * Try lowering the days late threshold
    * Include other risk scores besides 'high'
    * Consider other call statuses that might indicate difficulty in reaching the customer

3. Check for any data inconsistencies or unexpected values in these fields.

4. Consider if there are other relevant factors that could indicate a need for an Ownership Test.

In [24]:
# Analyze the distribution of each criterion separately
# 1. Customers with days late >= 60
eligible_days_late = df_merged[df_merged['days_late'] >= 60]

# 2. Customers with a 'high' risk score
eligible_high_risk = df_merged[df_merged['risk_Score'] == 'high']

# 3. Customers with a 'No Answer' call status
eligible_no_answer = df_merged[df_merged['call_status'] == 'No Answer']

# Print the counts and first few rows for each criterion
print("Eligible customers with days late >= 60:")
print(eligible_days_late.shape)
print(eligible_days_late.head())

print("\
Eligible customers with high risk score:")
print(eligible_high_risk.shape)
print(eligible_high_risk.head())

print("\
Eligible customers with No Answer call status:")
print(eligible_no_answer.shape)
print(eligible_no_answer.head())

# Check for data inconsistencies or unexpected values in these fields
print("\
Unique values in 'days_late':")
print(df_merged['days_late'].unique())

print("\
Unique values in 'risk_Score':")
print(df_merged['risk_Score'].unique())

print("\
Unique values in 'call_status':")
print(df_merged['call_status'].unique())

print("\
Done")

Eligible customers with days late >= 60:
(0, 14)
Empty DataFrame
Columns: [call_Date, loan_id, user_id, amount_to_repay_today, disbursement_date, days_late, risk_Score, amount_repaid, call_id, call_date, call_status, total_talk_time, first_disposition, second_disposition]
Index: []
Eligible customers with high risk score:
(2474, 14)
     call_Date   loan_id   user_id  amount_to_repay_today disbursement_date  \
44  2024-07-13  10080678   1856499                  37847        2023-04-25   
64  2024-07-13  10110787   8726730                  84899        2023-04-28   
153 2024-07-13  10214320   1761339                      0        2023-05-08   
213 2024-07-13  10293978  39931381                  24000        2023-05-16   
374 2024-07-13  10600063   1892147                      0        2023-06-15   

     days_late risk_Score  amount_repaid  call_id call_date call_status  \
44          17       high        6953.09      NaN       NaT         NaN   
64          18       high      476090.69

In [25]:
eligible_days_late

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition


In [26]:
eligible_high_risk

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition
44,2024-07-13,10080678,1856499,37847,2023-04-25,17,high,6953.09,NaN,NaT,NaN,NaN,NaN,NaN
64,2024-07-13,10110787,8726730,84899,2023-04-28,18,high,476090.69,NaN,NaT,NaN,NaN,NaN,NaN
153,2024-07-13,10214320,1761339,0,2023-05-08,16,high,5250.00,NaN,NaT,NaN,NaN,NaN,NaN
213,2024-07-13,10293978,39931381,24000,2023-05-16,16,high,500.00,NaN,NaT,NaN,NaN,NaN,NaN
374,2024-07-13,10600063,1892147,0,2023-06-15,20,high,1603.00,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,2024-07-13,13672023,16784100,0,2024-04-28,17,high,1600.00,293829990.0,2024-07-12,Answer,7.0,NoCNN,No Answer
19665,2024-07-13,13672169,49827836,20390,2024-04-28,25,high,10.00,NaN,NaT,NaN,NaN,NaN,NaN
19676,2024-07-13,13672345,45287208,12966,2024-04-28,25,high,834.50,291057703.0,2024-07-10,Answer,7.0,NoCNN,No Answer
19677,2024-07-13,13672345,45287208,12966,2024-04-28,25,high,834.50,289816908.0,2024-07-09,Answer,59.0,RPC,PTP Follow Up


In [27]:
eligible_no_answer

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition


### Based on these findings, we can conclude that:

1. The 'days_late' field has a maximum value of 30, which explains why we found no customers with days late >= 60. We may need to adjust this criterion.

2. The 'risk_Score' field has the expected values: 'low', 'medium', and 'high'.

3. The 'call_status' field does not have a 'No Answer' status. Instead, it has 'Answer', 'Abandoned in outgoing queue', 'Infinite Preview (no call)', and some  values.

#### To refine our Ownership Test eligibility criteria, we should consider the following adjustments:

1. Lower the 'days_late' threshold to a value within the observed range, such as 25 or 30 days.
2. Keep the 'high' risk score criterion as it is.
3. Redefine the call status criterion to include 'Abandoned in outgoing queue' and 'Infinite Preview (no call)' as these might indicate difficulty in reaching the customer.

In [29]:
# Implementing the adjusted criteria for Ownership Test eligibility
# New criteria:
# 1. Days late >= 30
# 2. Risk Score = 'high'
# 3. Call status = 'Abandoned in outgoing queue' or 'Infinite Preview (no call)'

adjusted_eligible = df_merged[(df_merged['days_late'] >= 30) & 
                               (df_merged['risk_Score'] == 'high') & 
                               (df_merged['call_status'].isin(['Abandoned in outgoing queue', 'Infinite Preview (no call)']))]

# Print the count and first few rows of the adjusted eligible population
print("Adjusted eligible customers for Ownership Test:")
print(adjusted_eligible.shape)
print(adjusted_eligible.head())

# Define the objective of each experiment, target audience, and other experiment-related details
# Objective: To identify customers eligible for Ownership Test based on adjusted criteria
# Target Audience: Customers with high risk scores and specific call statuses
# Test-Control Population: Test group will be the adjusted eligible customers; control group will be the remaining customers
# Timelines: 1 month for initial outreach and follow-up

# Print the experiment details
experiment_details = {
    'Objective': 'Identify customers eligible for Ownership Test based on adjusted criteria',
    'Target Audience': 'Customers with high risk scores and specific call statuses',
    'Test-Control Population': 'Test group: adjusted eligible customers; Control group: remaining customers',
    'Timelines': '1 month for initial outreach and follow-up'
}

print("\
Experiment Details:")
for key, value in experiment_details.items():
    print(f'{key}: {value}')

# List of information/data points that could help improve categorization
improvement_data_points = [
    'Customer demographics (age, income, etc.)',
    'Loan history (previous loans, repayment behavior)',
    'Communication history (response rates, preferred contact methods)',
    'Payment history (frequency, amounts, missed payments)',
    'External credit scores or reports',
    'Behavioral data (website/app usage patterns)',
    'Geographic data (location-based risk factors)',
    'Social media or public records information'
]

print("\
Data Points for Improvement:")
for point in improvement_data_points:
    print(f'- {point}')

print("\
Done")

Adjusted eligible customers for Ownership Test:
(0, 14)
Empty DataFrame
Columns: [call_Date, loan_id, user_id, amount_to_repay_today, disbursement_date, days_late, risk_Score, amount_repaid, call_id, call_date, call_status, total_talk_time, first_disposition, second_disposition]
Index: []
Experiment Details:
Objective: Identify customers eligible for Ownership Test based on adjusted criteria
Target Audience: Customers with high risk scores and specific call statuses
Test-Control Population: Test group: adjusted eligible customers; Control group: remaining customers
Timelines: 1 month for initial outreach and follow-up
Data Points for Improvement:
- Customer demographics (age, income, etc.)
- Loan history (previous loans, repayment behavior)
- Communication history (response rates, preferred contact methods)
- Payment history (frequency, amounts, missed payments)
- External credit scores or reports
- Behavioral data (website/app usage patterns)
- Geographic data (location-based risk fac

In [30]:
adjusted_eligible

,call_Date,loan_id,user_id,amount_to_repay_today,disbursement_date,days_late,risk_Score,amount_repaid,call_id,call_date,call_status,total_talk_time,first_disposition,second_disposition


### Experiment Details:
###### Objective:
   * Identify customers eligible for Ownership Test based on adjusted criteria.
   * To assess the effectiveness of targeted outreach strategies on customer engagement and repayment rates.
   
##### Target Audience:
   * Customers with high risk scores and specific call statuses.
   * Customers identified as high-risk based on their risk scores and previous call statuses.
   
##### Test-Control Population: Test group will consist of adjusted eligible customers; the control group will include the remaining customers.
###### Timelines:
   * 1 month for initial outreach and follow-up.
   * 2 months for further outreach and follow-up analysis.

##### Methodology:
1. Segment the customer base into high-risk and low-risk groups.
2. Develop tailored communication strategies for each segment.
3. Implement a multi-channel outreach approach (calls, emails, SMS).
4. Monitor engagement metrics (response rates, follow-up actions).
5. Analyze repayment behavior post-outreach.

##### Expected Outcomes:
   * Increased engagement from high-risk customers.
   * Improved repayment rates among targeted customers.
   * Insights into the effectiveness of different communication channels.
   * Data-driven recommendations for future outreach strategies.

##### Metrics for Success:
   * Engagement rates (response to outreach).
   * Repayment rates (amount repaid vs. amount owed).
   * Customer feedback on communication effectiveness.
   * Comparison of outcomes between high-risk and low-risk groups.

#### These details outline a comprehensive approach to assess the effectiveness of targeted outreach strategies on customer engagement and repayment rates.

### Data Points for Improvement:
##### To further enhance the categorization process, the following information/data points could be beneficial:
1. Customer demographics (age, income, etc.)
2. Loan history (previous loans, repayment behavior)
3. Communication history (response rates, preferred contact methods)
4. Payment history (frequency, amounts, missed payments)
5. External credit scores or reports
6. Behavioral data (website/app usage patterns)
7. Geographic data (location-based risk factors)
8. Social media or public records information